In [3]:
import pandas as pd
import numpy as np
import urllib2
import datetime as dt
import matplotlib.pyplot as plt
 
def get_google_data(symbol, period, window):
    url_root = 'http://www.google.com/finance/getprices?i='
    url_root += str(period) + '&p=' + str(window)
    url_root += 'd&f=d,o,h,l,c,v&df=cpct&q=' + symbol
    response = urllib2.urlopen(url_root)
    data = response.read().split('\n')
    #actual data starts at index = 7
    #first line contains full timestamp,
    #every other line is offset of period from timestamp
    parsed_data = []
    anchor_stamp = ''
    end = len(data)
    for i in range(7, end):
        cdata = data[i].split(',')
        if 'a' in cdata[0]:
            #first one record anchor timestamp
            anchor_stamp = cdata[0].replace('a', '')
            cts = int(anchor_stamp)
        else:
            try:
                coffset = int(cdata[0])
                cts = int(anchor_stamp) + (coffset * period)
                parsed_data.append((dt.datetime.fromtimestamp(float(cts)), float(cdata[1]), float(cdata[2]), float(cdata[3]), float(cdata[4]), float(cdata[5])))
            except:
                pass # for time zone offsets thrown into data
    df = pd.DataFrame(parsed_data)
    df.columns = ['ts', 'o', 'h', 'l', 'c', 'v']
    df.index = df.ts
    del df['ts']
    return df

In [5]:
spy = get_google_data('SPY', 300, 10)

In [6]:
spy.head()

,o,h,l,c,v
ts,,,,,
2016-07-11 08:35:00,213.49,213.50,213.150,213.21,761721.0
2016-07-11 08:40:00,213.52,213.59,213.420,213.49,473852.0
2016-07-11 08:45:00,213.57,213.66,213.430,213.53,471931.0
2016-07-11 08:50:00,213.46,213.67,213.435,213.58,453161.0
2016-07-11 08:55:00,213.39,213.49,213.360,213.47,323097.0
